<a href="https://colab.research.google.com/github/KushagraGoel2024/Quote-analysis-author-and-sentiment/blob/main/Author_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow scikit-learn pandas numpy pickle5
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM, Bidirectional, SimpleRNN
import pickle5 as pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255314 sha256=b28151964248adf9a02349736fe29d0efafb3c486b0fc2849ec9f89eb7f1e24e
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/quotes.csv')
df = df[['quote', 'author']]
print(df)
df = df.sample(frac=1).reset_index(drop=True)

                                                  quote                 author
0        “Be yourself; everyone else is already taken.”            Oscar Wilde
1     “I'm selfish, impatient and a little insecure....         Marilyn Monroe
2     “Two things are infinite: the universe and hum...        Albert Einstein
3                      “So many books, so little time.”            Frank Zappa
4     “A room without books is like a body without a...  Marcus Tullius Cicero
...                                                 ...                    ...
2500  “Morality is simply the attitude we adopt towa...           Oscar Wilde 
2501  “Don't aim at success. The more you aim at it ...      Viktor E. Frankl 
2502  “In life, finding a voice is speaking and livi...           John Grisham
2503  “Winter is the time for comfort, for good food...          Edith Sitwell
2504                      “Silence is so freaking loud”          Sarah Dessen 

[2505 rows x 2 columns]


In [ ]:
tokenizer = Tokenizer(num_words=500, oov_token='<OOV>')
tokenizer.fit_on_texts(df['author'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['quote'])
padded_sequences = pad_sequences(sequences, maxlen=500, truncating='post')
authorlabels = pd.get_dummies(df['author']).values
print(pd.get_dummies(df['author']))

      A. A. Milne   A.A. Milne  A.A. Milne   Abigail Van Buren  \
0                0           0            0                  0   
1                0           0            0                  0   
2                0           0            0                  0   
3                0           0            0                  0   
4                0           0            0                  0   
...            ...         ...          ...                ...   
2500             0           0            0                  0   
2501             0           0            0                  0   
2502             0           0            0                  0   
2503             0           0            0                  0   
2504             0           0            0                  0   

      Abraham Lincoln  Adolf Hitler  Adrienne Rich  Agatha Christie  \
0                   0             0              0                0   
1                   0             0              0               

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, authorlabels, test_size=0.3)
print(x_train.shape)

(1753, 500)


In [ ]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=500))
model.add(SimpleRNN(200,return_sequences=True))
model.add(SimpleRNN(200,return_sequences=True))
model.add(Bidirectional(LSTM(200)))
model.add(Dense(878, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          500000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 200)          60200     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 500, 200)          80200     
                                                                 
 bidirectional (Bidirectiona  (None, 400)              641600    
 l)                                                              
                                                                 
 dense (Dense)               (None, 878)               352078    
                                                                 
Total params: 1,634,078
Trainable params: 1,634,078
Non-trainable params: 0
______________________________________________

In [ ]:
model.fit(x_train, y_train, epochs=50 , batch_size=100, validation_data=(x_test, y_test))

Epoch 1/50
18/18 [==============================] - 168s 9s/step - loss: 6.4516 - accuracy: 0.0211 - val_loss: 6.3208 - val_accuracy: 0.0479
Epoch 2/50
18/18 [==============================] - 160s 9s/step - loss: 6.0234 - accuracy: 0.0337 - val_loss: 6.4820 - val_accuracy: 0.0479
Epoch 3/50
18/18 [==============================] - 157s 9s/step - loss: 5.9637 - accuracy: 0.0365 - val_loss: 6.5997 - val_accuracy: 0.0173
Epoch 4/50
18/18 [==============================] - 160s 9s/step - loss: 5.9403 - accuracy: 0.0302 - val_loss: 6.6559 - val_accuracy: 0.0479
Epoch 5/50
18/18 [==============================] - 156s 9s/step - loss: 5.9289 - accuracy: 0.0302 - val_loss: 6.7091 - val_accuracy: 0.0479
Epoch 6/50
18/18 [==============================] - 158s 9s/step - loss: 5.9247 - accuracy: 0.0308 - val_loss: 6.7376 - val_accuracy: 0.0479
Epoch 7/50
18/18 [==============================] - 162s 9s/step - loss: 5.9226 - accuracy: 0.0331 - val_loss: 6.7825 - val_accuracy: 0.0479
Epoch 8/50
18

In [ ]:
model.save('/content/gdrive/MyDrive/author_analysis_model.h5')

In [ ]:
import keras
model = keras.models.load_model('/content/gdrive/MyDrive/author_analysis_model.h5')


In [ ]:
def predict_sentiment(text):
    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=500)
    return model.predict(text_sequence)

In [ ]:
for n in range(2505):
  text_input =(df.iloc[n]['quote'])
  predicted_entiment = predict_sentiment(text_input)
  print(predicted_entiment)

Streaming output truncated to the last 5000 lines.
  0.00108527 0.00101072 0.00101756 0.00120803 0.001108   0.00113507
  0.00112299 0.00126907 0.00106579 0.0011757  0.00108965 0.00104316
  0.00132717 0.00117305 0.00103872 0.00117641 0.00109143 0.00105185
  0.00142206 0.00094712 0.0012463  0.00105598 0.00113139 0.00113674
  0.00108438 0.00106211 0.00097295 0.00100134 0.00125199 0.00116837
  0.00099558 0.00105096 0.00117214 0.00115125 0.00144072 0.00121748
  0.00125519 0.00101748 0.00114952 0.00101658 0.00115306 0.00104613
  0.00117618 0.00121752 0.00112246 0.00117674 0.00123067 0.00106812
  0.00118355 0.00099786 0.00106184 0.00107394 0.00113882 0.00106734
  0.00123368 0.0013779  0.00106146 0.00101058 0.00101514 0.00100613
  0.00128043 0.0011227  0.00114231 0.00097855 0.00098785 0.00142915
  0.00117795 0.00108789 0.00124148 0.00116156 0.00091226 0.0012011
  0.00100837 0.00108007 0.00104049 0.00095667 0.00110285 0.0010211
  0.00108169 0.00113569 0.00117181 0.00119888 0.00109297 0.00119792